In this notebook we show how to prepare all the input and parameter files necessary to run Diva4D.<br>
The code rely on the classes defined for 2D (module `pydiva2d`) and 4D (module `pydiva4d`). 

In [4]:
import os
import sys
import logging
import shutil
import subprocess
import datetime
from importlib import reload

In [5]:
sys.path.insert(0, "../")
import pydiva4d

Reload the module (during for development), (un)comment if necessary.

In [6]:
# reload(pydiva4d)

# User inputs and configuration

Configure logging

In [7]:
pydiva4d.logger.handlers[0].setLevel(logging.DEBUG)
pydiva4d.logger.handlers[1].setLevel(logging.DEBUG)

Indicate the path to the Diva installation you want to work with (ending with `diva-x.y.z`).

In [8]:
divabasedir = "/home/ctroupin/Software/DIVA/DIVA-diva-4.7.1"

## Create the sub-directories and files

In [48]:
divadirs = pydiva4d.Diva4DDirectories(divabasedir)

2017-04-21 10:48:18,732 - pydiva4d - INFO - Diva 4D input directory: /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/input
2017-04-21 10:48:18,734 - pydiva4d - INFO - Diva 4D output directory: /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/output/
2017-04-21 10:48:18,739 - pydiva4d - INFO - Diva 4D mesh directory: /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/newinput/divamesh/
2017-04-21 10:48:18,741 - pydiva4d - INFO - Diva 4D parameter directory: /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/newinput/divaparam/
2017-04-21 10:48:18,742 - pydiva4d - INFO - Diva 4D field directory: /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/output/3Danalysis/Fields


In [49]:
diva2Dfiles = pydiva4d.Diva2Dfiles(divadirs.diva2d)

In [50]:
diva4Dfiles = pydiva4d.Diva4Dfiles(divadirs.diva4d)

2017-04-21 10:48:23,223 - pydiva4d - INFO - Creating Diva 4D file names and paths
2017-04-21 10:48:23,226 - pydiva4d - INFO - datasource file:   /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/datasource
2017-04-21 10:48:23,228 - pydiva4d - INFO - constandrefe file: /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/constandrefe
2017-04-21 10:48:23,230 - pydiva4d - INFO - driver file:       /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/driver
2017-04-21 10:48:23,231 - pydiva4d - INFO - monthlist file:    /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/monthlist
2017-04-21 10:48:23,232 - pydiva4d - INFO - qflist file:       /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/qflist
2017-04-21 10:48:23,234 - pydiva4d - INFO - varlist file:      /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/varlist
2017-04-21 10:48:23,235 - pydiva4d - INFO - yearlist file:     /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4

## List of user parameters

Set the different parameters necessary for a 4D analysis.<br>
For clarity, they are separated by input files.

### List of ODV spreadsheet files

An example data file is distributed with the Diva code (*Example4D* directory): 

In [16]:
datasource = pydiva4d.Datasource(os.path.join(divabasedir, "Example4D/blacksea_data_CTD.txt"))

2017-04-21 12:28:55,875 - pydiva4d - INFO - Creating Diva 4D Datasource object
2017-04-21 12:28:55,880 - pydiva4d - WARNING - Argument converted from string to list


### Parameters for the advection constraint and the reference field

In [17]:
advection_flag = 0               # indicates if advection is activated
ref_flag = 0                     # indicates if there is a reference field
var_year_code = '00000000'       # variable year code
var_month_code = '0000'          # variable month code

In [18]:
constandrefe = pydiva4d.Constandrefe(advection_flag, ref_flag, var_year_code, var_month_code)

2017-04-21 12:29:06,770 - pydiva4d - INFO - Creating Diva 4D Constandrefe object


### Parameters for the 4D analysis

In [20]:
extraction_flag = 1              # Data extraction: 1 do it, 0 do nothing, -1 press coord, -10 pressure+Saunders
coast_flag = 1                   # Boundary lines and coastlines generation: 0 nothing, 1: contours, 2: UV, 3: 1+2
clean_flag = 0                   # Cleaning data on mesh: 1, 2: RL, 3: both, 4: 1 + outliers elimination, 5: =4+2
min_datanum = 0                  # Minimal number of data in a layer. If less, uses data from any month.
param_flag = -30                 # Parameters estimation and vertical filtering
min_l, max_l = 1.4, 10           # Minimal/maximal correlation length during optimization
min_snr, max_snr = 0.1, 5.0      # Minimal/maximal signal-to-noise ratio during optimization
analysisref_flag = 1             # Creation of reference field: 2 do reference, 1 do analysis and 0 do nothing
lower_level, upper_level = 1, 4  # Depth lower/upper level numbers
netcdf4d_flag = 1                # 4D netcdf files: 0 (no generation) or 1
gnuplot_flag = 0                 # Gnuplots generation: 0 (no generation) or 1
detrend_groupnum = 0             # Data detrending: number of groups, 0 if no detrending.

In [21]:
driver = pydiva4d.Driver(extraction_flag, coast_flag, clean_flag, min_datanum, param_flag, min_l,\
                               max_l, min_snr, max_snr, analysisref_flag, lower_level,\
                               upper_level, netcdf4d_flag, gnuplot_flag, detrend_groupnum)

2017-04-21 12:29:23,577 - pydiva4d - INFO - Creating Diva 4D Driver object


### Parameters for the netCDF file

In [22]:
# Title string for 3D NetCDF file
ncdf_title = '4D Test on the Black Sea using Jupyter notebook'
# Reference time for data (ie: days since since 1900-01-01), if not climatological data
ncdf_reftime = 'months since since xxxx-01-01'
# Time value (that reprsents the data set), if not climatological data
ncdf_timeval = 1200
# Cell_methode string
ncdf_cellmethod = 'time: mean (this month data from all years)'
# Institution name: where the dataset was produced.
ncdf_institution = 'University of Liege, GeoHydrodynamics and Environment Research'
# Production group and e-mail
ncdf_groupemail = 'Diva group. E-mails : a.barth@ulg.ac.be ; swatelet@ulg.ac.be ; ctroupin@ulg.ac.be'
# Source (observation, radiosonde, database, model-generated data,...)
ncdf_source = 'data_from various sources for diva software testing work'
ncdf_comment = 'No comment'
ncdf_authoremail = 'Name@institution'
ncdf_acknowlegment = 'No acknowledgement'

In [23]:
netcdfinfo = pydiva4d.Ncdfinfo(ncdf_title, ncdf_reftime, ncdf_timeval, ncdf_cellmethod,\
                                     ncdf_institution, ncdf_groupemail, ncdf_source,\
                                     ncdf_comment, ncdf_authoremail, ncdf_acknowlegment)

2017-04-21 10:31:13,577 - pydiva4d - INFO - Creating Diva 4D Ncdfinfo object


### Parameters for the output grid

In [24]:
xmin, ymin = 27.0, 40.0        # longitute and latitude of the lower-left corner of the domain
nx, ny = 151, 76               # number of cells in the zonal and meridional directions
dx, dy = 0.1, 0.1              # spatial steps in the zonal and meridional directions

### Parameters for the analysis

In [25]:
CorrelationLength = 1.5
SignalToNoiseRatio = 0.5
VarianceBackgroundField = 1.0
ExclusionValue = -99.
iCoordChange = 2
iSpec = 11
iReg = 2

In [26]:
parameters2D = pydiva4d.Diva2DParameters(CorrelationLength, iCoordChange, iSpec, iReg, xmin, ymin, dx, dy, nx, ny,
                                ExclusionValue, SignalToNoiseRatio, VarianceBackgroundField)

## Others: monthlist, QC flags, variables, years, contour levels

In [27]:
months = pydiva4d.Monthlist(['0101', '0202', '0303'])       # list of strings representing the periods (months)
qflags = pydiva4d.Qflist([0, 1])                            # list of integers representing the accepted flags
variables = pydiva4d.Varlist(['Temperature', 'Salinity'])   # list of strings representing the variables to extract
years = pydiva4d.Yearlist(['19002009',])                    # list of strings representing the perior (years)
contours = pydiva4d.Contourdepth([30., 20., 10., 0.])       # list of floats representing the vertical levels

2017-04-21 10:32:43,390 - pydiva4d - INFO - Creating Diva 4D Monthlist object
2017-04-21 10:32:43,392 - pydiva4d - INFO - Creating Diva 4D Qflist object
2017-04-21 10:32:43,394 - pydiva4d - INFO - Creating Diva 4D Varlist object
2017-04-21 10:32:43,396 - pydiva4d - INFO - Creating Diva 4D Yearlist object
2017-04-21 10:32:43,397 - pydiva4d - INFO - Creating Diva 4D Contourdepth object
2017-04-21 10:32:43,398 - pydiva4d - DEBUG - Ordering list of depths in decreasing order


# Create the new input files

Now we have set all the parameters, we can write the information into the corresponding `Diva` files.<br>
All the objects have a *write_to* method that is used to write the information to a file.

## Write the different input files

In [28]:
datasource.write_to(diva4Dfiles.datasource)

2017-04-21 10:33:15,697 - pydiva4d - INFO - Written into file /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/datasource


In [29]:
constandrefe.write_to(diva4Dfiles.constandrefe)

2017-04-21 10:33:18,240 - pydiva4d - INFO - Written into file /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/constandrefe


In [30]:
driver.write_to(diva4Dfiles.driver)

2017-04-21 10:33:18,888 - pydiva4d - INFO - Written into file /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/driver


In [31]:
months.write_to(diva4Dfiles.monthlist)

2017-04-21 10:33:19,308 - pydiva4d - INFO - Written in file /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/monthlist


In [32]:
qflags.write_to(diva4Dfiles.qflist)

2017-04-21 10:33:19,635 - pydiva4d - INFO - Written in file /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/qflist


In [33]:
variables.write_to(diva4Dfiles.varlist)

2017-04-21 10:33:21,538 - pydiva4d - INFO - Written in file /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/varlist


In [34]:
years.write_to(diva4Dfiles.yearlist)

2017-04-21 10:33:21,753 - pydiva4d - INFO - Written into file /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/yearlist


In [35]:
netcdfinfo.write_to(diva4Dfiles.ncdfinfo)

2017-04-21 10:33:21,942 - pydiva4d - INFO - Written into file /home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/JRA4/Climatology/ncdfinfo


In [36]:
parameters2D.write_to(diva4Dfiles.param)

# Run divadoall (4D analysis)

We call the main script `divadoall` to perform the 4D analysis.<br>
The time for the analysis depends on the number of variables, levels, periods, etc, so can expect it to run for a few minutes with the default values set in this example.

In [37]:
pydiva4d.logger.info("Starting to run 'divadoall'")
divadoall_script = './divadoall'
env = os.environ.copy()
env["PATH"] = ".:" + env["PATH"]

p = subprocess.Popen(divadoall_script, stdout=subprocess.PIPE, cwd=divadirs.diva4d, env=env)
out = p.stdout.read()
pydiva4d.logger.info("Finished 'divadoall'")

2017-04-21 10:33:48,848 - pydiva4d - INFO - Starting to run 'divadoall'


Finally we write the `Diva` execution log into the same log file:

In [38]:
pydiva4d.logger.info("Diva execution log:")
with open(pydiva4d.logfile, 'w') as f:
    f.write(str(out).replace('\\n', '\n'))

2017-04-21 10:39:45,262 - pydiva4d - INFO - Diva execution log:


In [39]:
pydiva4d.logger.info("Finished the 'divadoall' run")

2017-04-21 10:40:01,682 - pydiva4d - INFO - Finished the 'divadoall' run


In [3]:
pydiva4d.logfile

NameError: name 'pydiva4d' is not defined

## Data visualisation

### ncview

If you have the [ncview](http://meteora.ucsd.edu/%7Epierce/ncview_home_page.html) tool installed, you can have a quick look to the different layers, periods and variables to ensure the `Diva` execution was fine.

In [53]:
Temperature4Dfile = os.path.join(divadirs.diva4doutput, "3Danalysis/Temperature.19002009.4Danl.nc")

True